In [16]:
import psycopg2
import csv
def con():
    return psycopg2.connect(
        host="localhost",
        database="lab10pgadmin",
        user="postgres",
        password="sayasaya"
    )
def insert_consol(conn):
   name = input("Enter name: ")
   phone = input("Enter phone: ")
   with conn:
        with conn.cursor() as cur:
            cur.execute("INSERT INTO phonebook (name, phone) VALUES ( %s, %s)", 
                        (name, phone))
            print("INSERTED")
def insert_csv(path):
    with con() as conn:
        with conn.cursor() as cur:
            with open(path, 'r') as file:
                reader = csv.reader(file)
                next(reader)  
                for row in reader:
                    name, phone = row
                    cur.execute("INSERT INTO phonebook (name,  phone) VALUES ( %s, %s)", 
                                (name,  phone))
                    print(f"INSERTED {name} with phone {phone}")
def query(name):
    with con()as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM phonebook WHERE name=%s", (name,))
            print( cur.fetchall())
def delete(name):
    with con()as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM phonebook WHERE name=%s", (name,))
            print("DELETED")
def create_table():
    with con() as conn:
        with conn.cursor() as cur:
            cur.execute("DROP TABLE IF EXISTS phonebook")
            cur.execute("""
                CREATE TABLE phonebook (
                    name TEXT NOT NULL,
                    phone TEXT NOT NULL
                )
            """)
            print("Table 'phonebook' ensured to exist.")

def menu():
    create_table()
    print("1. Insert phonebook")
    print("2. Insert csv")
    print("3. Query phonebook")
    print("4. Delete phonebook")
    print("5. Exit")
    choice = input("Enter your choice: ")
    if choice == '1':
        conn = con()
        insert_consol(conn)
    elif choice == '2':
        path = input("Enter the path to the CSV file: ")
        insert_csv(path)
    elif choice == '3':
        name = input("Enter name to query: ")
        query(name)
    elif choice == '4':
        name = input("Enter name to delete: ")
        delete(name)
    elif choice == '5':
        exit()
    else:
        print("Invalid choice")
  
if __name__ == "__main__":
    menu()
            

Table 'phonebook' ensured to exist.
1. Insert phonebook
2. Insert csv
3. Query phonebook
4. Delete phonebook
5. Exit
DELETED


In [1]:
import pygame
import random
import time
import psycopg2

# Initialize Pygame
pygame.init()

# Database connection
def create_connection():
    return psycopg2.connect(database="lab10pgadmin", user="postgres", password="sayasaya", host="localhost", port="5432")

def create_tables(conn):
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS "User" (
        user_id SERIAL PRIMARY KEY,
        username VARCHAR(100) UNIQUE NOT NULL
    );

    CREATE TABLE IF NOT EXISTS user_score (
        score_id SERIAL PRIMARY KEY,
        user_id INT REFERENCES "User"(user_id),
        level INT NOT NULL,
        score INT NOT NULL
    );
    """)
    conn.commit()

def get_user_id(conn, username):
    cur = conn.cursor()
    cur.execute("SELECT user_id FROM \"User\" WHERE username = %s", (username,))
    result = cur.fetchone()
    
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO \"User\" (username) VALUES (%s) RETURNING user_id", (username,))
        user_id = cur.fetchone()[0]
        conn.commit()
        return user_id

def insert_user_score(conn, user_id, level, score):
    cur = conn.cursor()
    cur.execute("INSERT INTO user_score (user_id, level, score) VALUES (%s, %s, %s)", (user_id, level, score))
    conn.commit()

# Game settings
WIDTH, HEIGHT = 600, 400
GRID = 25
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()

# Snake and food logic
snake = [(100, 100), (90, 100), (80, 100)]
direction = (GRID, 0)

score = 0
level = 1
speed = 5
game_over = False

class Food:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.color = RED
        self.points = 1
        self.spawn_time = 0
        self.life = 0
    
    def new(self):
        while True:
            self.x = random.randint(0, (WIDTH//GRID)-1)*GRID
            self.y = random.randint(0, (HEIGHT//GRID)-1)*GRID
            if (self.x, self.y) not in snake:
                break
        
        if random.random() < 0.2:
            self.color = BLUE
            self.points = 3
            self.life = 5
        else:
            self.color = RED
            self.points = 1
            self.life = 0
        
        self.spawn_time = time.time()
    
    def check_time(self):
        if self.life > 0 and (time.time()-self.spawn_time) > self.life:
            return True
        return False

food = Food()
food.new()

def message(msg, color):
    font = pygame.font.Font(None, 48)
    mesg = font.render(msg, True, color)
    screen.blit(mesg, [WIDTH//2-100, HEIGHT//2-24])

# Main Game Loop
def game_loop(username):
    conn = create_connection()
    user_id = get_user_id(conn, username)

    global score, level, speed, game_over
    snake = [(100, 100), (90, 100), (80, 100)]
    direction = (GRID, 0)
    score = 0
    level = 1
    speed = 5
    game_over = False

    food = Food()
    food.new()

    while not game_over:
        screen.fill(WHITE)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and direction != (0, GRID):
                    direction = (0, -GRID)
                elif event.key == pygame.K_DOWN and direction != (0, -GRID):
                    direction = (0, GRID)
                elif event.key == pygame.K_LEFT and direction != (GRID, 0):
                    direction = (-GRID, 0)
                elif event.key == pygame.K_RIGHT and direction != (-GRID, 0):
                    direction = (GRID, 0)
        
        new_x = snake[0][0] + direction[0]
        new_y = snake[0][1] + direction[1]
        new_head = (new_x, new_y)
        
        if new_x < 0 or new_x >= WIDTH or new_y < 0 or new_y >= HEIGHT or new_head in snake:
            game_over = True
        
        snake.insert(0, new_head)
        
        if new_head == (food.x, food.y):
            score += food.points
            if score % 5 == 0:
                level += 1
                speed += 2
            food.new()
        else:
            snake.pop()
        
        if food.check_time():
            food.new()
        
        # Drawing snake and food
        pygame.draw.rect(screen, food.color, (food.x, food.y, GRID, GRID))
        for part in snake:
            pygame.draw.rect(screen, GREEN, (part[0], part[1], GRID, GRID))
        
        # Displaying score and level
        font = pygame.font.Font(None, 24)
        text = font.render(f"Score: {score} Level: {level}", True, BLACK)
        screen.blit(text, (10, 10))
        
        if game_over:
            message("GAME OVER!", RED)
            insert_user_score(conn, user_id, level, score)
            conn.close()

        pygame.display.flip()
        clock.tick(speed)

def main():
    conn = create_connection()
    create_tables(conn)
    
    username = input("Enter your username: ")
    game_loop(username)
    
    conn.close()
    print("Database connection closed.")

if __name__ == "__main__":
    main()
    pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Database connection closed.
